In [4]:
import numpy as np
import pandas as pd
import mne

In [2]:
data = mne.io.read_raw_edf('./eeg_34/eeg34_reduced1.edf', preload=True)

Extracting EDF parameters from /home/anjali/Desktop/Mehnat/BTP2/eeg_34/eeg34_reduced1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 255  =      0.000 ...     0.996 secs...


In [5]:
df = data.to_data_frame()
type(df)

Converting "time" to "<class 'numpy.int64'>"...


pandas.core.frame.DataFrame

In [7]:
df_np = data[:][0]
df_np.shape[1]

256

In [5]:
# C = np.fft.fftn(df_np)
# sum(C[0:10]).shape

In [6]:
import numpy as np
fs = 256
# data in numpy format

eeg_bands = {'Delta': (0, 4),
                 'Theta': (4, 8),
                 'Alpha': (8, 12),
                 'Beta': (12, 30),
                 'Gamma': (30, 45)}
def psd(data):
    fs = 256                                
    N = data.shape[1] # total num of points 
    print(N)

    # Get real amplitudes of FFT (only in postive frequencies)
    fft_vals = np.absolute(np.fft.rfft(data))

    # FORMULA from some online post
    # psd_vals = (1/(fs*N)) * xdft * xdft;

    n_rows = fft_vals.shape[0]
    n_cols = fft_vals.shape[1]
    psd_vals = np.zeros(shape=(n_rows, n_cols))

    for i in range(n_rows):
        for j in range(n_cols):
            psd_vals[i][j] = (N/fs) * fft_vals[i][j] * fft_vals[i][j];

    print(psd_vals.shape)

    # Get frequencies for amplitudes in Hz
    fft_freq = np.fft.rfftfreq(data.shape[1], 1.0/fs)

    # Define EEG bands
    eeg_bands = {'Delta': (0, 4),
                 'Theta': (4, 8),
                 'Alpha': (8, 12),
                 'Beta': (12, 30),
                 'Gamma': (30, 45)}

    # Take the mean of the fft amplitude for each EEG band
    eeg_band_fft = dict()
    for band in eeg_bands:  
        freq_ix = np.where((fft_freq >= eeg_bands[band][0]) & 
                           (fft_freq <= eeg_bands[band][1]))[0]
        eeg_band_fft[band] = np.mean(psd_vals[:,freq_ix])
    return eeg_band_fft

    # Plot the data (using pandas here cause it's easy)
#     import pandas as pd
#     df = pd.DataFrame(columns=['band', 'val'])
#     df['band'] = eeg_bands.keys()
#     df['val'] = [eeg_band_fft[band] for band in eeg_bands]
#     ax = df.plot.bar(x='band', y='val', legend=False)
#     ax.set_xlabel("EEG band")
#     ax.set_ylabel("Mean PSD")

#     for band in eeg_bands:
#         print(eeg_band_fft[band])

In [2]:
import csv

In [11]:
data = mne.io.read_raw_edf('./eeg_34/eeg34_reduced1.edf', preload=True)
data = data[:][0]
psd_vals = psd(data)
for val in psd_vals:
    print(psd_vals[val] * 1000000)

Extracting EDF parameters from /home/anjali/Desktop/Mehnat/BTP2/eeg_34/eeg34_reduced1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 255  =      0.000 ...     0.996 secs...
256
(21, 129)
33.5078234198
4.23481412049
1.96249091334
1.09095973512
0.345930132248


In [7]:
with open('eeg73-power.csv', 'w') as file:
    writer = csv.writer(file)
    for row in range(20):
        row_to_add = []
        data = mne.io.read_raw_edf('Splitted Files/eeg_73/73.{}.edf'.format(row+1), preload=True)
        data = data[:][0]
        psd_vals = psd(data)
        for band in eeg_bands:
            row_to_add.append(psd_vals[band] * 1000000)  
        writer.writerow(row_to_add)

Extracting EDF parameters from /home/anjali/Desktop/Mehnat/BTP2/Splitted Files/eeg_73/73.1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 255  =      0.000 ...     0.996 secs...
256
(21, 129)
Extracting EDF parameters from /home/anjali/Desktop/Mehnat/BTP2/Splitted Files/eeg_73/73.2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 255  =      0.000 ...     0.996 secs...
256
(21, 129)
Extracting EDF parameters from /home/anjali/Desktop/Mehnat/BTP2/Splitted Files/eeg_73/73.3.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 255  =      0.000 ...     0.996 secs...
256
(21, 129)
Extracting EDF parameters from /home/anjali/Desktop/Mehnat/BTP2/Splitted Files/eeg_73/73.4.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 255  =      0.000 ...     0.996 secs...
256
(21, 129)
Extr